In [1]:
# Get L2 Revenue and post it to a database (csv in github for now)
import pandas as pd
import sys
import time
import json
sys.path.append("../helper_functions")
import web3py_utils as w3py
import duneapi_utils as du
from web3 import Web3
from datetime import datetime
sys.path.pop()

import os

In [2]:
# https://github.com/ethereum-optimism/optimism/blob/b86522036ad11a91de1d1dadb6805167add83326/specs/predeploys.md?plain=1#L50

# Name, contract, Proxy ABI, Implementation ABI
fee_vaults = [
    ['SequencerFeeVault','0x4200000000000000000000000000000000000011'
        # ,'[{"inputs":[{"internalType":"address","name":"_admin","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"previousAdmin","type":"address"},{"indexed":false,"internalType":"address","name":"newAdmin","type":"address"}],"name":"AdminChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"implementation","type":"address"}],"name":"Upgraded","type":"event"},{"stateMutability":"payable","type":"fallback"},{"inputs":[],"name":"admin","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_admin","type":"address"}],"name":"changeAdmin","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"implementation","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_implementation","type":"address"}],"name":"upgradeTo","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_implementation","type":"address"},{"internalType":"bytes","name":"_data","type":"bytes"}],"name":"upgradeToAndCall","outputs":[{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"payable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
        # ,'[{"inputs":[{"internalType":"address","name":"_recipient","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"address","name":"from","type":"address"}],"name":"Withdrawal","type":"event"},{"inputs":[],"name":"MIN_WITHDRAWAL_AMOUNT","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"RECIPIENT","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"l1FeeWallet","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalProcessed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdraw","outputs":[],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
        ],
    ['BaseFeeVault','0x4200000000000000000000000000000000000019'
        # ,'[{"inputs":[{"internalType":"address","name":"_admin","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"previousAdmin","type":"address"},{"indexed":false,"internalType":"address","name":"newAdmin","type":"address"}],"name":"AdminChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"implementation","type":"address"}],"name":"Upgraded","type":"event"},{"stateMutability":"payable","type":"fallback"},{"inputs":[],"name":"admin","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_admin","type":"address"}],"name":"changeAdmin","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"implementation","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_implementation","type":"address"}],"name":"upgradeTo","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_implementation","type":"address"},{"internalType":"bytes","name":"_data","type":"bytes"}],"name":"upgradeToAndCall","outputs":[{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"payable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
        # ,'[{"inputs":[{"internalType":"address","name":"_recipient","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"address","name":"from","type":"address"}],"name":"Withdrawal","type":"event"},{"inputs":[],"name":"MIN_WITHDRAWAL_AMOUNT","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"RECIPIENT","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalProcessed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdraw","outputs":[],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
        ],
    ['L1FeeVault','0x420000000000000000000000000000000000001A'
        # ,'[{"inputs":[{"internalType":"address","name":"_admin","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"previousAdmin","type":"address"},{"indexed":false,"internalType":"address","name":"newAdmin","type":"address"}],"name":"AdminChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"implementation","type":"address"}],"name":"Upgraded","type":"event"},{"stateMutability":"payable","type":"fallback"},{"inputs":[],"name":"admin","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_admin","type":"address"}],"name":"changeAdmin","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"implementation","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_implementation","type":"address"}],"name":"upgradeTo","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_implementation","type":"address"},{"internalType":"bytes","name":"_data","type":"bytes"}],"name":"upgradeToAndCall","outputs":[{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"payable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
        # ,'[{"inputs":[{"internalType":"address","name":"_recipient","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"},{"indexed":false,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"address","name":"from","type":"address"}],"name":"Withdrawal","type":"event"},{"inputs":[],"name":"MIN_WITHDRAWAL_AMOUNT","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"RECIPIENT","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalProcessed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"withdraw","outputs":[],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
        ],
]

# Aiming to eventually read from superchain-resitry + some non-superchain static adds
chains_rpcs = pd.read_csv('inputs/chain_metadata.csv').values.tolist()
print(chains_rpcs)

[['op mainnet', ' https://mainnet.optimism.io', ' ETH', ' ethereum', ' yes'], ['base', ' https://mainnet.base.org/', ' ETH', ' ethereum', ' yes'], ['zora network', ' https://rpc.zora.energy/', ' ETH', ' ethereum', ' yes'], ['public goods network', ' https://rpc.publicgoods.network/', ' ETH', ' ethereum', ' yes'], ['aevo network', ' https://api.aevo.xyz', ' ETH', ' ethereum', ' no'], ['opbnb', ' https://opbnb-mainnet-rpc.bnbchain.org', ' BNB', ' bnb', ' no']]


In [3]:
# Calculate the method signature hash (this step is optional if you already know the hash)
method_signature = "totalProcessed()"
method_id = Web3.keccak(text=method_signature)[:4].hex()
# Verify the method ID (this step is also optional, but useful for debugging)
print(f"Method ID: {method_id}")

block_number = 1 # replace with your desired block number

Method ID: 0x84411d65


In [4]:
data_arr = []

for chain in chains_rpcs:
    chain_name = chain[0]
    rpc = chain[1]
    gas_token = chain[2]
    da_layer = chain[3]
    is_superchain_registry = chain[4]

    try:
        w3_conn = Web3(Web3.HTTPProvider(rpc))
        # Get the timestamp of the latest block
        block_timestamp = w3_conn.eth.get_block('latest').timestamp
        block_number = w3_conn.eth.get_block('latest').number
        # Convert the UNIX timestamp to a human-readable format
        block_time = datetime.utcfromtimestamp(block_timestamp).strftime('%Y-%m-%d %H:%M:%S')

        for vault in fee_vaults:
            vault_name = vault[0]
            vault_address = vault[1]
            # vault_abi = json.loads(vault[2])
            # implementation_abi = json.loads(vault[3])
            # Call the function directly using eth_call
            response = w3_conn.eth.call({
                'to': vault_address,
                'data': method_id
            })
            wei_balance = w3_conn.eth.get_balance(vault_address)
            # Decode the result (assuming the function returns a uint256)
            proxy_processed_wei = Web3.to_int(hexstr=response.hex())
            
            alltime_revenue_wei = proxy_processed_wei+wei_balance
            alltime_revenue_native = alltime_revenue_wei/1e18

            print(chain_name + ' | ' + vault_name + ': ' \
                + str(proxy_processed_wei) + ' | bal: ' + str(wei_balance)\
                + ' | total eth: ' + str( (alltime_revenue_native) )
                )
            
            tmp = pd.DataFrame(
                    [[block_time, block_number, chain_name, vault_name, vault_address, alltime_revenue_native, gas_token, da_layer, is_superchain_registry]]
                    ,columns =['block_time','block_number','chain_name','vault_name','vault_address','alltime_revenue_native','gas_token','da_layer','is_superchain_registry']
                    )
            data_arr.append(tmp)
            time.sleep(1)
    except:
        continue

data_df = pd.concat(data_arr)

op mainnet | SequencerFeeVault: 1063857200132073207047 | bal: 109326973996315601597 | total eth: 1173.1841741283888
op mainnet | BaseFeeVault: 222912724643847907088 | bal: 20392493836299784566 | total eth: 243.3052184801477
op mainnet | L1FeeVault: 3399092561901994272487 | bal: 309336338723387708329 | total eth: 3708.428900625382
base | SequencerFeeVault: 1592801383437017825498 | bal: 40952371978868248420 | total eth: 1633.7537554158862
base | BaseFeeVault: 314161722455325783336 | bal: 34452712700799383389 | total eth: 348.6144351561252
base | L1FeeVault: 1025424229958661395932 | bal: 45390785723705809357 | total eth: 1070.8150156823672
zora network | SequencerFeeVault: 156833819511677884835 | bal: 328580996071941092 | total eth: 157.16240050774982
zora network | BaseFeeVault: 0 | bal: 19695625923426 | total eth: 1.9695625923426e-05
zora network | L1FeeVault: 118743562764297720407 | bal: 8767306578428756680 | total eth: 127.51086934272647
public goods network | SequencerFeeVault: 0 | b

In [5]:
# display(data_df)

# Assuming data_df is your dataframe
file_path = 'outputs/all_time_revenue_data.csv'

# Check if the file exists
if os.path.exists(file_path):
    # If file exists, append without writing the header
    data_df.to_csv(file_path, mode='a', header=False, index=False)
else:
    # If file doesn't exist, create it and write the header
    data_df.to_csv(file_path, mode='w', header=True, index=False)

In [6]:
# Write to Dune Table

dune_df = pd.read_csv(file_path)

display(dune_df)

# Write to Dune
du.write_dune_api_from_pandas(dune_df, 'op_stack_chains_daily_cumulative_revenue',\
                             'All-Time (cumulative) revenue for fee vaults on OP Stack Chains. Pulled from RPCs')

,block_time,block_number,chain_name,vault_name,vault_address,alltime_revenue_native,gas_token,da_layer,is_superchain_registry
0,2023-09-05 16:28:19,109166261,op mainnet,SequencerFeeVault,0x4200000000000000000000000000000000000011,1.173184e+03,ETH,ethereum,yes
1,2023-09-05 16:28:19,109166261,op mainnet,BaseFeeVault,0x4200000000000000000000000000000000000019,2.433052e+02,ETH,ethereum,yes
2,2023-09-05 16:28:19,109166261,op mainnet,L1FeeVault,0x420000000000000000000000000000000000001A,3.708429e+03,ETH,ethereum,yes
3,2023-09-05 16:28:23,3570978,base,SequencerFeeVault,0x4200000000000000000000000000000000000011,1.633754e+03,ETH,ethereum,yes
4,2023-09-05 16:28:23,3570978,base,BaseFeeVault,0x4200000000000000000000000000000000000019,3.486144e+02,ETH,ethereum,yes
5,2023-09-05 16:28:23,3570978,base,L1FeeVault,0x420000000000000000000000000000000000001A,1.070815e+03,ETH,ethereum,yes
6,2023-09-05 16:28:27,3618734,zora network,SequencerFeeVault,0x4200000000000000000000000000000000000011,1.571624e+02,ETH,ethereum,yes
7,2023-09-05 16:28:27,3618734,zora network,BaseFeeVault,0x4200000000000000000000000000000000000019,1.969563e-05,ETH,ethereum,yes
8,2023-09-05 16:28:27,3618734,zora network,L1FeeVault,0x420000000000000000000000000000000000001A,1.275109e+02,ETH,ethereum,yes
9,2023-09-05 16:28:31,2412292,public goods network,SequencerFeeVault,0x4200000000000000000000000000000000000011,4.973594e+00,ETH,ethereum,yes


table at: dune_upload.op_stack_chains_daily_cumulative_revenue
Response status code: 200
Response content: b'{"success":true}'
